# DREAM data reduction

In [ ]:
import scipp as sc
import scippneutron as scn
import sciline
from ess import dream, powder
from ess.powder.types import *
from ess.dream.io.geant4 import providers as geant4_providers

## Define reduction parameters

We define a dictionary containing the reduction parameters.
The keys are types defined in [essdiffraction.types](../generated/modules/ess.diffraction.types.rst).

In [ ]:
params = {
    Filename[SampleRun]: dream.data.simulated_diamond_sample(),
    Filename[VanadiumRun]: dream.data.simulated_vanadium_sample(),
    Filename[EmptyCanRun]: dream.data.simulated_empty_can(),
    NeXusDetectorName: "mantle",
    # The upper bounds mode is not yet implemented.
    UncertaintyBroadcastMode: UncertaintyBroadcastMode.drop,
    # Edges for binning in d-spacing
    DspacingBins: sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom"),
    # Mask in time-of-flight to crop to valid range
    TofMask: lambda x: (x < sc.scalar(0.0, unit="ns"))
    | (x > sc.scalar(86e6, unit="ns")),
    TwoThetaMask: None,
    WavelengthMask: None,
}

# Not available in simulated data
sample = sc.DataGroup(position=sc.vector([0.0, 0.0, 0.0], unit="mm"))
params[RawSample[SampleRun]] = sample
params[RawSample[VanadiumRun]] = sample

source = sc.DataGroup(position=sc.vector([-3.478, 0.0, -76550], unit="mm"))
params[RawSource[SampleRun]] = source
params[RawSource[VanadiumRun]] = source

charge = sc.scalar(1.0, unit="µAh")
params[AccumulatedProtonCharge[SampleRun]] = charge
params[AccumulatedProtonCharge[VanadiumRun]] = charge

## Create pipeline using Sciline

We use the `powder` and `geant4` providers to build our pipeline.

In [ ]:
providers = (
    *geant4_providers,
    *powder.providers,
)

pipeline = sciline.Pipeline(providers, params=params)
pipeline = powder.with_pixel_mask_filenames(pipeline, [])

We can visualize the graph for computing the final normalized result for intensity as a function of d-spacing:

In [ ]:
pipeline.visualize(IofDspacing, graph_attr={"rankdir": "LR"})

We then call `compute()` to compute the result:

In [ ]:
result = pipeline.compute(IofDspacing)
result

In [ ]:
dspacing_histogram = result.hist()
dspacing_histogram.plot()

We can now save the result to disk:

In [ ]:
dspacing_histogram.coords["dspacing"] = sc.midpoints(
    dspacing_histogram.coords["dspacing"]
)
scn.io.save_xye("dspacing.xye", dspacing_histogram)

## Compute intermediate results

For inspection and debugging purposes, we can also compute intermediate results.
To avoid repeated computation (including costly loading of files), we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
intermediates = pipeline.compute(
    (
        DataWithScatteringCoordinates[SampleRun],
        MaskedData[SampleRun],
    )
)

intermediates[DataWithScatteringCoordinates[SampleRun]]

In [ ]:
intermediates[MaskedData[SampleRun]].bins.concat().hist(
    two_theta=300, wavelength=300
).plot(norm="log")

## Grouping by scattering angle

The above pipeline focuses the data by merging all instrument pixels to produce a 1d d-spacing curve.
If instead we want to group into $2\theta$ bins, we can alter the pipeline parameters by adding some binning in $2\theta$:

In [ ]:
pipeline[TwoThetaBins] = sc.linspace(
    dim="two_theta", unit="rad", start=0.8, stop=2.4, num=17
)

In [ ]:
grouped_dspacing = pipeline.compute(IofDspacingTwoTheta)
grouped_dspacing

In [ ]:
angle = sc.midpoints(grouped_dspacing.coords["two_theta"])
sc.plot(
    {
        f"{angle[group].value:.3f} {angle[group].unit}": grouped_dspacing[
            "two_theta", group
        ].hist()
        for group in range(2, 6)
    }
)

In [ ]:
grouped_dspacing.hist().plot(norm="log")